# Aula - Avaliando modelos

Vamos ver outras estratégias para tentar escolher modelos por algum tipo de previsão do nosso erro de generalização.

- 1) Holdout
- 2) Leave One Out Cross Validation (LOOCV)
- 3) Cross Validation (CV)

## 1) Estratégia "Holdout set": Conjuntos de treino, validação e teste

Como vimos, no aprendizado de máquina nós temos alguns dados (__conjunto de treino__), e depois fazemos um experimento com uma amostra de dados que nunca vimos (__conjunto de teste__) para saber o quão bem o modelo consegue generalizar.

Assim, temos o erro dentro do conjunto de treino, $E_{in}$, e o erro de generalização, pra dados daquele tipo fora desse conjunto, $E_{out}$. 
<br><br>

<div>
    <img src="images/treino_teste.png" width=500>
</div>

O problema é que, __se usarmos o conjunto de teste de qualquer forma para aprendizado, o erro que obtivermos nele deixa de refletir o erro de generalização__. 

Por exemplo, se treinarmos 3 modelos, e compararmos eles usando o conjunto de teste, o erro no teste não reflete mais o $E_{out}$.

Outro exemplo são certas transformações dos nossos dados. Imagina que pegamos nossos dados, "normalizamos" eles (ou seja, pegamos nossas features e transformamos elas de forma que tenham um range de 0 a 1), e então fazemos a divisão entre conjunto de treino e conjunto de teste. Nesse caso, você já usou o conjunto de teste para "aprender" algo (para normalizar, a gente usa o maior valor da feature na tabela). Logo, sua medida de $E_{out}$ não vale mais. 

O que fazer então? Nós usamos o __conjunto de validação__ (ou _hold-out set_).
<br><br>
<div>
    <img src="images/treino_validacao.png" width=500>
</div>

Ao separar uma amostra (conjunto de validação), e usá-la apenas para seleção de modelos, nós assumimos que esse uso não afeta muito o erro. Então vamos dizer que o erro de validação, $E_{val}$, __aproxima de certa forma o erro de generalização__.

<img src="images/cv.png" width=500 text="https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch06/ch06.ipynb">


Isso foi o que fizemos nas últimas aulas.

Após validar e escolher o modelo, nós queremos sempre seguir adiante com o melhor modelo possível. Assim, para melhorar o modelo, nós aumentamos o número de dados para treinar o modelo final.

Então aí sim, nós juntamos treino, validação e teste em uma única base, e treinamos o modelo final. Entende-se que os erros do nosso algoritmo só tendem a diminuir, quando fazemos isso.

## 2) Leave One Out Cross Validation (LOOCV)

Nós usamos, até agora, a estratégia de "hold-out set" (ou de conjunto de validação) para podermos comparar modelos.

Porém, ela tem algumas fraquezas. Especificamente, a gente precisa de dados suficientes no conjunto de validação para tentar aproximar melhor o erro de generalização. Mas isso faz com que tenhamos cada vez menos dados de treino, para treinar o melhor modelo possível!

Será que teria alguma forma de garantirmos uma boa validação, mas ainda tendo o máximo possível de dados pra treino? 

A resposta é __sim__. Entra em cena o __Leave One Out Cross Validation (LOOCV)__.
<br><br>
<div>
    <img src="images/LOOCV.png" width=600>
</div>
<br>

Ele funciona assim:
- Nós tiramos 1 ponto dos dados de treino
- Treinamos o modelo nos outros N-1 pontos
- Avaliamos o modelo naquele ponto que tiramos
- Repete esse procedimento para _todos_ os pontos da base de treino 

Embora o Scikit-Learn tenha ferramentas para usarmos o LOOCV, raramente esse método é aplicado na prática. O motivo disso é que ele é __muito custoso computacionalmente__, e esse custo aumenta cada vez mais quanto mais dados de treino tivermos.

Além disso, em termos de acurácia o LOO geralmente resulta em uma maior variância. Dado que os folds construídos são basicamente identicos uns aos outros (e idêntinco ao dataset original), não há diferença significativa entre os modelos criados em cada iteração.

Nós não vamos testar esse método devido à esses motivos e vamos focar no seu primo mais útil: a validação cruzada.

[Documentação do sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html#sklearn.model_selection.LeaveOneOut)

## 3) Cross Validation (CV)

ref: Material de aulas do prof. Sandro Saorin

O __Cross Validation (CV)__ (ou validação cruzada, em português) é uma técnica muito utilizada para estimar o erro de generalização do modelo. Ele é semelhante ao LOOCV, mas invés de separarmos apenas 1 ponto de cada vez, nós separamos um __bloco de pontos__.

Com ela podemos testar a capacidade de generalização de um modelo. A técnica faz divisões na base de dados de treino e teste e permite treinar e validar seus dados em diversos grupos distintos. Dessa forma, conseguimos mensurar a flexibilidade ou capacidade de generalização de um modelo antes mesmo da chegada de novos dados.

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width=600>

Vamos discutir o passo a passo do que está acontecendo:
- Como sempre, primeiro temos que garantir que a validação não aconteça usando a base de teste.
- A gente separa a nossa base de treino em diversas partes (o mais comum são 3, 5 ou 10).
- Cada parte vai ser usada 1 vez como base de validação, para um treino realizado nas outras partes. (Na figura, por exemplo, temos 5 rodadas, e em cada rodada um bloco diferente é usado como validação, enquanto os outros 4 são usados para treino).
- Calculamos as métricas de avaliação em cada uma dessas rodadas, para a base de validação.
- Por fim, temos como resultado o score médio para o nosso modelo (tirando a média das métricas para cada parte usada como validação), ou seja, o quão bom ele está generalizando.

<br>

No sklearn podemos criar esses _folds_ utilizando os médotos [__K-Fold__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) e [__Stratified K-Fold__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html?highlight=stratifiedkfold#sklearn.model_selection.StratifiedKFold). Eles irão retornar os indices dos arrays de dados de acordo com o número de partições escolhidas.

A diferença entre amobos é semelhante ao que discutimos sobre usar ou não o parâmetro "stratify" da função "train_test_split". O __K-Fold__ faz uma quebra sem se preocupar com a distribuição das classes, enquanto o __Stratified K-Fold__ garante a mesma proporção em todos os _folds_.

A gente pode usar o _stratified k-fold_ para quaisquer problemas que quisermos, embora o mais comum seja ele ser usado quando as nossas classes _target_ são muito __desbalanceadas__. Se uma das classes aparece muito menos que a outra, é possível que tenhamos uma quebra muito ruim. Em casos extremos, se uma classe aparece muito pouco, é possível que, sem a estratificação, ela sequer apareça em alguns _folds_.

<br><br>

Vamos ver como funciona a criação e uso dos folds na prática.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
with open('../data/bank-names.txt', 'r') as fp:
    print(fp.read())

Citation Request:
  This dataset is public available for research. The details are described in [Moro et al., 2011]. 
  Please include this citation if you plan to use this database:

  [Moro et al., 2011] S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

  Available at: [pdf] http://hdl.handle.net/1822/14838
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2011-esm-1.txt

1. Title: Bank Marketing

2. Sources
   Created by: Paulo Cortez (Univ. Minho) and Sérgio Moro (ISCTE-IUL) @ 2012
   
3. Past Usage:

  The full dataset was described and analyzed in:

  S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European S

In [5]:
# Carregando o dataset e olhando para os nossos dados
df = pd.read_csv('../data/bank-full.csv', sep=',')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [6]:
# Vamos dar uma olhada em como está os nossos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  Target     45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [8]:
# Verificando a proporção da target
df['Target'].value_counts()

no     39922
yes     5289
Name: Target, dtype: int64

In [10]:
# Dropando a target e a duração da chamada das nossas features
X = df.drop(columns=['Target', 'duration'])

As nossas variáveis categóricas precisam ser transformadas em dados numéricos.
Até agora, no máximo transformamos em um número de 0 a N.

Contundo, geralmente essas variáveis não tem uma ordem. ('assalariado' é maior ou menor do que 'aposentado'?) Quando colocamos 0 a N, nós sem querer introduzimos essa ordem artificialmente.

Assim, muitas vezes o ideal é quebrar aquela coluna em variáveis "indicadoras".

Vão ser várias colunas, com valor 0 ou 1, sendo 0 quando não é aquele valor, e 1 quando é aquele valor. Chamamos isso de __One Hot Encoding__.

In [11]:
# Vamos ver um exemplo na prática.
pontos = pd.DataFrame({'a':[1, 2, 1, 2, 1, 2],
                       'b':[0.45, 0.1, 0.23, 0.98, 0.1, 0.999]})

print("Como era nosso dataframe original:")
display(pontos)

print("Como ele fica após fazermos One Hot Encoding:")
display(pd.get_dummies(pontos, prefix_sep='_', columns=['a']))

Como era nosso dataframe original:


,a,b
0,1,0.450
1,2,0.100
2,1,0.230
3,2,0.980
4,1,0.100
5,2,0.999


Como ele fica após fazermos One Hot Encoding:


,b,a_1,a_2
0,0.450,1,0
1,0.100,0,1
2,0.230,1,0
3,0.980,0,1
4,0.100,1,0
5,0.999,0,1


O sklearn também tem uma ferramenta para realizar o mesmo procedimento, que é uma classe chamada "OneHotEncoding", mas não usaremos ele nessa aula. 

In [12]:
# Fazendo um get_dummies para colunar as nossas variáveis categóricas
X_with_dummies = pd.get_dummies(X, prefix_sep = '_', drop_first=True, columns=['job', 
                                                                                'marital', 
                                                                                'education',
                                                                                'default',
                                                                                'housing',
                                                                                'loan',
                                                                                'contact',
                                                                                'month',
                                                                                'poutcome'])

In [14]:
# transformando a target
y_target = np.where(df['Target'] == 'yes', 1, 0)
y_target

array([0, 0, 0, ..., 1, 0, 0])

In [15]:
#Separando em train e test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_with_dummies, 
                                                    y_target, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                    stratify = y_target)

In [16]:
# Testando com o modelo DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Instancia o modelo
model = DecisionTreeClassifier(random_state = 42)

In [35]:
# Importa o StratifiedKFold do sklearn.model_selection
from sklearn.model_selection import StratifiedKFold

# Instancia a classe do StratifiedKFold
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

3

In [31]:
# Vamos transformar o X de treino em array do numpy para facilitar manipulação
X_train_arr = X_train.values

In [36]:
# Neste caso, teremos que montar manualmente cada iteração.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

list_accuracy = []
list_precision = []
list_recall = []
list_f1_score = []

i = 1
for train_index, val_index in kf.split(X_train, y_train):
    
    print("============================================================================================")
    print("Fold ", i)
    print("TRAIN:", train_index, "VALIDATION:", val_index)
    
    # Pegando o X e o y de treino e de validação para aquela iteração
    KFold_X_train, KFold_X_val = X_train_arr[train_index], X_train_arr[val_index]
    KFold_y_train, KFold_y_val = y_train[train_index], y_train[val_index]
    
    # Treinando o modelo da iteração
    model.fit(KFold_X_train, KFold_y_train)
    
    # Fazendo as previsões no "fold" de validação
    y_pred = model.predict(KFold_X_val)
    
    #Calcula as métricas
    acc = accuracy_score(KFold_y_val, y_pred)
    prec = precision_score(KFold_y_val, y_pred)
    recall = recall_score(KFold_y_val, y_pred)
    f1 = f1_score(KFold_y_val, y_pred)
    
    # Mostrando em tela as métricas
    print("Accuracy: ", round(acc, 2))
    print("Precison: ", round(prec, 2))
    print("Recal:    ", round(recall,2))
    print("F1-Score: ", round(f1, 2))
    
    # Salvando as métricas na lista
    list_accuracy.append(acc)
    list_precision.append(prec)
    list_recall.append(recall)
    list_f1_score.append(f1)
    i += 1
print("============================================================================================")

Fold  1
TRAIN: [    0     1     2 ... 31642 31645 31646] VALIDATION: [    4     7    12 ... 31638 31643 31644]
Accuracy:  0.82
Precison:  0.28
Recal:     0.33
F1-Score:  0.3
Fold  2
TRAIN: [    1     3     4 ... 31643 31644 31646] VALIDATION: [    0     2     5 ... 31640 31641 31645]
Accuracy:  0.83
Precison:  0.29
Recal:     0.35
F1-Score:  0.32
Fold  3
TRAIN: [    0     2     4 ... 31643 31644 31645] VALIDATION: [    1     3     9 ... 31639 31642 31646]
Accuracy:  0.83
Precison:  0.27
Recal:     0.28
F1-Score:  0.28


## Cross_val_score: facilitando nossa vida

O Scikit-Learn já possui métodos implementados que calculam os score dentro dos folds e um deles é o  [__cross_val_score__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html): Precisamos apenas informar qual a quantidade de folds (cv) e qual métrica queremos avaliar (score). A saída do método é a média para cada partição considerando a métrica escolhida.

Se quisermos avaliar mais de uma métrica dentro dos mesmos folds ou se quisermos o tempo necessário para o fit podemos utilizar o método [__cross_validate__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

Se o parâmetro __cv__ for um inteiro ou None e o estimador for de classificação (binário ou multiclass), o StratifiedKFold será usado, caso contrário o KFold é o selecionado.

In [26]:
# Utilizando o Cross Validation
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5)
scores

array([0.82369668, 0.83191153, 0.83156897, 0.82730289, 0.8277769 ])

In [29]:
print("A acurácia média de todos os folds é %0.2f com um desvio padrão de %0.3f" % (scores.mean(), scores.std()))

A acurácia média de todos os folds é 0.83 com um desvio padrão de 0.003


In [39]:
from sklearn.model_selection import cross_validate
nome_metricas = ['accuracy', 'precision', 'recall']
metricas = cross_validate(model, X_train, y_train, cv=5, scoring=nome_metricas)

for met in metricas:
    print(f"- {met}:")
    print(f"-- {metricas[met]}")
    print(f"-- {np.mean(metricas[met])} +- {np.std(metricas[met])}\n")  

- fit_time:
-- [0.14157557 0.13328838 0.1261704  0.13502955 0.13005805]
-- 0.13322439193725585 +- 0.005153436336664123

- score_time:
-- [0.00476956 0.00502563 0.0047617  0.00493693 0.00513077]
-- 0.004924917221069336 +- 0.00014383108192928104

- test_accuracy:
-- [0.82369668 0.83191153 0.83156897 0.82730289 0.8277769 ]
-- 0.8284513949055189 +- 0.00303555954853931

- test_precision:
-- [0.27863046 0.30135301 0.29262087 0.2978236  0.28015075]
-- 0.29011573793604795 +- 0.009199735014163364

- test_recall:
-- [0.31848853 0.33063428 0.31081081 0.35135135 0.30135135]
-- 0.32252726410621146 +- 0.01731217848681251



In [40]:
metricas

{'fit_time': array([0.14157557, 0.13328838, 0.1261704 , 0.13502955, 0.13005805]),
 'score_time': array([0.00476956, 0.00502563, 0.0047617 , 0.00493693, 0.00513077]),
 'test_accuracy': array([0.82369668, 0.83191153, 0.83156897, 0.82730289, 0.8277769 ]),
 'test_precision': array([0.27863046, 0.30135301, 0.29262087, 0.2978236 , 0.28015075]),
 'test_recall': array([0.31848853, 0.33063428, 0.31081081, 0.35135135, 0.30135135])}

## Exercícios

Faça a validação cruzada para três métricas distintas no dataset bank-full e compare o KNN com a Decision Tree.

## Bibliografia e Aprofundamento
- [Métricas](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
- [Time Series Split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html#sklearn.model_selection.TimeSeriesSplit)
- [Sklearn Cross validation iterators](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators)